# Combine Census PEP characteristics data for Ohio counties

This script will retrieve the County Population by Characteristics datasets for several decades from the Census Population Estimates Program website, merge them, and standarize them as a long-format CSV file. 

## Setup

### Import required packages

In [280]:
import os
import pandas as pd
import re
import json
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### Parameters

In [281]:
# Define county groups
CONST_REGIONS = {
    "15-County Region": ["Delaware", "Fairfield", "Fayette", "Franklin", "Hocking", "Knox", "Licking", "Logan", "Madison", "Marion", "Morrow", "Perry", "Pickaway", "Ross", "Union"],
    "10-County Region": ["Delaware", "Fairfield", "Franklin", "Knox", "Licking", "Madison", "Marion", "Morrow", "Pickaway", "Union"],
    "CORPO Region": ["Fairfield", "Knox", "Madison", "Marion", "Morrow", "Pickaway", "Union"]
}

# Define input and output directories
INPUT_DIR = "./input_data"
OUTPUT_DIR = "./output_data"

### Define inputs

#### Census PEP data 2000+

In [282]:
CENSUSPEP_COUNTY_WIDE_2000_TABLE_FILENAME = "censusPep_county_char.csv"
CENSUSPEP_COUNTY_WIDE_2000_TABLE_PATH = os.path.join(INPUT_DIR, CENSUSPEP_COUNTY_WIDE_2000_TABLE_FILENAME)
CENSUSPEP_COUNTY_WIDE_2000_TABLE_SCHEMA_FILENAME = CENSUSPEP_COUNTY_WIDE_2000_TABLE_FILENAME.replace(".csv","_schema.json")
CENSUSPEP_COUNTY_WIDE_2000_TABLE_SCHEMA_PATH = os.path.join(INPUT_DIR, CENSUSPEP_COUNTY_WIDE_2000_TABLE_SCHEMA_FILENAME)

#### Census PEP data 1980-1989

In [283]:
CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_FILENAME = "pe-02.csv"
CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_PATH = os.path.join(INPUT_DIR, CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_FILENAME)
CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_SCHEMA_FILENAME = CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_FILENAME.replace(".csv","_schema.json")
CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_SCHEMA_PATH = os.path.join(INPUT_DIR, CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_SCHEMA_FILENAME)

#### Census PEP data 1990-1999

In [284]:
CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_FILENAME = "stch-icen1990.txt"
CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_PATH = os.path.join(INPUT_DIR, CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_FILENAME)
CENSUSPEP_COUNTY_WIDE_1990_1999_CSV_TABLE_FILENAME = "stch-icen1990.csv"
CENSUSPEP_COUNTY_WIDE_1990_1999_CSV_TABLE_PATH = os.path.join(INPUT_DIR, CENSUSPEP_COUNTY_WIDE_1990_1999_CSV_TABLE_FILENAME)
CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_SCHEMA_FILENAME = CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_FILENAME.replace(".txt","_schema.json")
CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_SCHEMA_PATH = os.path.join(INPUT_DIR, CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_SCHEMA_FILENAME)

### Define output

In [312]:
OUTPUT_TABLE_FILENAME = "output_data.csv"
OUTPUT_TABLE_PATH = os.path.join(OUTPUT_DIR, OUTPUT_TABLE_FILENAME)
OUTPUT_TABLE1_FILENAME = "output_data1.csv"
OUTPUT_TABLE1_PATH = os.path.join(OUTPUT_DIR, OUTPUT_TABLE1_FILENAME)

## Getting 2000+ data

In [286]:
# Adjust the path to where the .csv table and .json schema are located
table_path = "../morpc-censuspep-fetch/output_data/censusPep_county_char.csv"

# Read the CSV file into a DataFrame
data = pd.read_csv(table_path, index_col=0)

# Filter out rows where CTYNAME is not in the "10-County Region"
filtered_df = data[data['CTYNAME'].isin(CONST_REGIONS["10-County Region"])]

# Filter rows where "Variable" is equal to "TOT_POP"
filtered_df = filtered_df[filtered_df['Variable'] == 'TOT_POP']

# Save the census data to the input directory
filtered_df.to_csv(CENSUSPEP_COUNTY_WIDE_2000_TABLE_PATH)

In [287]:
grouped_sum = filtered_df.groupby(['YEAR', 'AGEGRP'])['Value'].sum().reset_index()
aggregated_df = pd.DataFrame(columns=['YEAR', 'AGEGRP', 'Value'])
aggregated_df = pd.concat([aggregated_df, grouped_sum], ignore_index=True)

# Show the aggregated DataFrame
aggregated_df.head()

aggregated_df.to_csv(CENSUSPEP_COUNTY_WIDE_2000_TABLE_PATH)

In [288]:
# Group by "YEAR" and sum "Value" for each group
yearly_sum = aggregated_df.groupby('YEAR')['Value'].sum().reset_index()

# Add a new column for "AGEGRP" with the value "Total" for these summary rows
yearly_sum['AGEGRP'] = 'Total'

# Rearrange columns to match the original DataFrame's order
yearly_sum = yearly_sum[['YEAR', 'AGEGRP', 'Value']]

# Append these new summary rows to the original DataFrame
updated_df = pd.concat([aggregated_df, yearly_sum], ignore_index=True)

# Convert AGEGRP to a type that sorts numerically first, then by string
updated_df['AGEGRP'] = pd.Categorical(updated_df['AGEGRP'], ordered=True, 
                                      categories=sorted(updated_df['AGEGRP'].unique(), key=str))

# Sort the DataFrame by 'YEAR' and then by 'AGEGRP'
sorted_df = updated_df.sort_values(by=['YEAR', 'AGEGRP'])

yearly_sum = yearly_sum[yearly_sum['YEAR'].isin([2000, 2010, 2020])]

# Filter the DataFrame to keep only rows where YEAR is 2000, 2010, or 2020
filtered_df = sorted_df[sorted_df['YEAR'].isin([2000, 2010, 2020])]

totals_df = filtered_df[filtered_df['AGEGRP'] == 'total']

In [289]:
yearly_sum.head()

,YEAR,AGEGRP,Value
0,2000,Total,1740458
10,2010,Total,3936535
20,2020,Total,6614062


In [290]:
AGEUNDER35 = ['0 to 4', '5 to 9', '10 to 14', '15 to 19', '20 to 24', '25 to 29', '30 to 34']

# Filter the DataFrame for the specified age groups
under_35_df = filtered_df[filtered_df['AGEGRP'].isin(AGEUNDER35)]

# Group by 'YEAR' and sum 'Value' for each group, creating a new 'AGEGRP' for "UNDER 35"
under_35_df = under_35_df.groupby('YEAR')['Value'].sum().reset_index()
under_35_df['AGEGRP'] = 'UNDER 35'

under_35_df.head()

,YEAR,Value,AGEGRP
0,2000,899510,UNDER 35
1,2010,1947657,UNDER 35
2,2020,3170280,UNDER 35


In [291]:
# Define the age groups for 35 to 64
AGE35TO64 = ['35 to 39', '40 to 44', '45 to 49','50 to 54','55 to 59','60 to 64']

# Filter the DataFrame for the specified age groups
filtered_df_35to64 = filtered_df[filtered_df['AGEGRP'].isin(AGE35TO64)]

# Group by 'YEAR' and sum 'Value' for each group, creating a new 'AGEGRP' for "35 TO 64"
age35to64_df = filtered_df_35to64.groupby('YEAR')['Value'].sum().reset_index()
age35to64_df['AGEGRP'] = '35 TO 64'

age35to64_df.head()

,YEAR,Value,AGEGRP
0,2000,662071,35 TO 64
1,2010,1560721,35 TO 64
2,2020,2522795,35 TO 64


In [292]:
# Define the age groups for 35 to 64
AGE65PLUS = ['65 to 69', '70 to 74', '75 to 79', '80 to 84', '85 or older'] 

# Filter the DataFrame for the specified age groups
over_64_df = filtered_df[filtered_df['AGEGRP'].isin(AGE65PLUS)]

# Group by 'YEAR' and sum 'Value' for each group, creating a new 'AGEGRP' for "35 TO 64"
over_64_df = over_64_df.groupby('YEAR')['Value'].sum().reset_index()
over_64_df['AGEGRP'] = '65 AND OLDER'

over_64_df.head()

,YEAR,Value,AGEGRP
0,2000,178877,65 AND OLDER
1,2010,428157,65 AND OLDER
2,2020,920987,65 AND OLDER


In [293]:
combined_2000_df = pd.concat([under_35_df, age35to64_df, over_64_df, yearly_sum], ignore_index=True)
combined_2000_df.head()

,YEAR,Value,AGEGRP
0,2000,899510,UNDER 35
1,2010,1947657,UNDER 35
2,2020,3170280,UNDER 35
3,2000,662071,35 TO 64
4,2010,1560721,35 TO 64


## Adding 1990

In [294]:
# Read the file using read_csv with a regex pattern as the delimiter for multiple spaces
df = pd.read_csv(CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_PATH, sep='\s+', header=None)

df.columns = ['YEAR', 'FIPSCOUNTY', 'AGEGRP', 'RACE-SEX', 'LATIN', 'Value']

df.to_csv(CENSUSPEP_COUNTY_WIDE_1990_1999_CSV_TABLE_PATH)

df.head()

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ogwynn\AppData\Local\Temp\ipykernel_10840\1969984832.py:2: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(CENSUSPEP_COUNTY_WIDE_1990_1999_TABLE_PATH, sep='\s+', header=None)


,YEAR,FIPSCOUNTY,AGEGRP,RACE-SEX,LATIN,Value
0,90,1001,0,1,1,239
1,90,1001,0,2,1,203
2,90,1001,1,1,1,821
3,90,1001,1,2,1,769
4,90,1001,2,1,1,1089


In [295]:
fips_codes_to_keep= [39041, 39045, 39049, 39083, 39089, 39097, 39101, 39117, 39129, 39159]

# Filter for 10-County region
filtered_df = df[df['FIPSCOUNTY'].isin(fips_codes_to_keep)]

# Keep only the 'YEAR', 'AGEGRP', and 'VALUE' columns
filtered_df = filtered_df[['YEAR', 'AGEGRP', 'Value']]

# Filter the DataFrame to keep only rows where YEAR equals 90
filtered_df = filtered_df[filtered_df['YEAR'] == 90]

print(filtered_df)

        YEAR  AGEGRP  Value
626240    90       0    482
626241    90       0    451
626242    90       1   2016
626243    90       1   1915
626244    90       2   2667
...      ...     ...    ...
644475    90      16      0
644476    90      17      0
644477    90      17      0
644478    90      18      0
644479    90      18      0

[3040 rows x 3 columns]


In [296]:
# Group by 'YEAR' and 'AGEGRP', and sum 'VALUE' for each group
added_1990_df = filtered_df.groupby(['YEAR', 'AGEGRP'], as_index=False)['Value'].sum()

# Print the combined DataFrame
print(added_1990_df)

    YEAR  AGEGRP   Value
0     90       0   23969
1     90       1   89499
2     90       2  107569
3     90       3  101009
4     90       4  111660
5     90       5  134734
6     90       6  141905
7     90       7  143236
8     90       8  126129
9     90       9  112327
10    90      10   83453
11    90      11   69314
12    90      12   61711
13    90      13   60062
14    90      14   52851
15    90      15   39541
16    90      16   29359
17    90      17   19221
18    90      18   15450


In [297]:
# Define the age groups for 35 to 64
AGE65PLUS = [14, 15, 16, 17, 18] 

# Filter the DataFrame for the specified age groups
over_64_df = added_1990_df[added_1990_df['AGEGRP'].isin(AGE65PLUS)]

# Group by 'YEAR' and sum 'Value' for each group, creating a new 'AGEGRP' for "35 TO 64"
over_64_df = over_64_df.groupby('YEAR')['Value'].sum().reset_index()
over_64_df['AGEGRP'] = '65 AND OLDER'

over_64_df.head()

,YEAR,Value,AGEGRP
0,90,156422,65 AND OLDER


In [298]:
# Define the age groups for 35 to 64
AGE35TO64 = [8, 9, 10,11,12,13]

# Filter the DataFrame for the specified age groups
filtered_df_35to64 = added_1990_df[added_1990_df['AGEGRP'].isin(AGE35TO64)]

# Group by 'YEAR' and sum 'Value' for each group, creating a new 'AGEGRP' for "35 TO 64"
age35to64_df = filtered_df_35to64.groupby('YEAR')['Value'].sum().reset_index()
age35to64_df['AGEGRP'] = '35 TO 64'

age35to64_df.head()

,YEAR,Value,AGEGRP
0,90,512996,35 TO 64


In [299]:
AGEUNDER35 = [0,1,2,3,4,5,6,7]

# Filter the DataFrame for the specified age groups
under_35_df = added_1990_df[added_1990_df['AGEGRP'].isin(AGEUNDER35)]

# Group by 'YEAR' and sum 'Value' for each group, creating a new 'AGEGRP' for "UNDER 35"
under_35_df = under_35_df.groupby('YEAR')['Value'].sum().reset_index()
under_35_df['AGEGRP'] = 'UNDER 35'

under_35_df.head()

,YEAR,Value,AGEGRP
0,90,853581,UNDER 35


In [300]:
# Sum all values in the "VALUE" column
total_value_sum = combined_1990_df['Value'].sum()

# Create a new DataFrame for the row to be added
new_row_df = pd.DataFrame({'YEAR': [90], 'AGEGRP': ['Total'], 'Value': [total_value_sum]})

combined_1990_df = pd.concat([under_35_df, age35to64_df, over_64_df, new_row_df], ignore_index=True)

# Change rows where YEAR="90" to YEAR="1990"
combined_1990_df.loc[combined_1990_df['YEAR'] == 90, 'YEAR'] = 1990

combined_1990_df.head()

,YEAR,Value,AGEGRP
0,1990,853581,UNDER 35
1,1990,512996,35 TO 64
2,1990,156422,65 AND OLDER
3,1990,9137994,Total


## Adding 1980

In [301]:
# Read the CSV file into a DataFrame
data = pd.read_csv(CENSUSPEP_COUNTY_WIDE_1980_1989_TABLE_PATH, index_col=0,low_memory=False)

# Remove the first four rows
df = data.iloc[4:]

# Use the first row to set the column names
df.columns = df.iloc[0]

# Drop the first row
df = df.drop(df.index[0])

# Reset the index
df.reset_index(drop=True, inplace=True)

# Keep only the first 18847 rows (years after 1980) of the DataFrame
df_filtered = df.iloc[:18847]


In [302]:
fips_codes_to_keep= ["39041", "39045", "39049", "39083", "39089", "39097", "39101", "39117", "39129", "39159"]

# Filter for 10-County region
filtered_df = df_filtered[df_filtered['FIPS State and County Codes'].isin(fips_codes_to_keep)]

filtered_df = filtered_df[['Under 5 years', '5 to 9 years', '10 to 14 years','15 to 19 years','20 to 24 years','25 to 29 years','30 to 34 years','35 to 39 years', '40 to 44 years','45 to 49 years','50 to 54 years','55 to 59 years','60 to 64 years','65 to 69 years','70 to 74 years','75 to 79 years','80 to 84 years','85 years and over']]

filtered_df.head()

Year of Estimate,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
12355,1972,2101,2319,2978,2417,2094,2113,1820,1576,1419,1352,1163,884,690,522,317,175,123
12356,1818,1966,2186,2763,2351,2138,2181,1937,1550,1341,1296,1206,979,857,722,552,378,347
12357,52,52,54,97,69,51,45,33,34,26,26,26,20,12,12,9,7,2
12358,50,38,66,118,62,43,41,23,31,27,26,28,23,25,16,8,7,6
12359,9,6,11,12,21,14,8,8,7,3,2,1,1,4,2,1,1,0


In [ ]:
# Convert all columns in the DataFrame to integer type
filtered_df = filtered_df.astype(int)

# Sum all columns individually
column_sums = filtered_df.sum()

# Sum the first 7 column sums
UNDER35SUM = column_sums.iloc[:7].sum()

# Sum the next 6 column sums (from the 8th to the 13th column)
AGE35TO64SUM = column_sums.iloc[7:13].sum()

# Sum the following 5 column sums after the first 13 (starting from the 14th column)
OVER64SUM = column_sums.iloc[13:18].sum()

TOTALSUM=(column_sums.sum())

# Create a new DataFrame for the row to be added
new_row_1980_1 = pd.DataFrame({'YEAR': [1980], 'AGEGRP': ['UNDER 35'], 'Value': [UNDER35SUM]})
# Create a new DataFrame for the row to be added
new_row_1980_2 = pd.DataFrame({'YEAR': [1980], 'AGEGRP': ['35 TO 64'], 'Value': [AGE35TO64SUM]})
# Create a new DataFrame for the row to be added
new_row_1980_3 = pd.DataFrame({'YEAR': [1980], 'AGEGRP': ['65 AND OLDER'], 'Value': [OVER64SUM]})
# Create a new DataFrame for the row to be added
new_row_1980_4 = pd.DataFrame({'YEAR': [1980], 'AGEGRP': ['Total'], 'Value': [TOTALSUM]})

combined_1980_df= pd.concat([new_row_1980_1, new_row_1980_2, new_row_1980_3, new_row_1980_4], ignore_index=True)

combined_1980_df.head()

## Combining Census data

In [304]:
combined_all_df= pd.concat([combined_1980_df, combined_1990_df, combined_2000_df], ignore_index=True)

# Sort the DataFrame by 'YEAR' and then by 'AGEGRP'
combined_all_df = combined_all_df.sort_values(by=['YEAR', 'AGEGRP'])

combined_all_df.to_csv(OUTPUT_TABLE_PATH)

## Getting County Projection data

In [307]:
# Path to the .xlsx file
excel_path = "../morpc-county-controls/deliverables/CountyControls_WEB.xlsx"

# Use ExcelFile to open the Excel file and iterate through sheets
with pd.ExcelFile(excel_path) as xls:
    for sheet_name in xls.sheet_names:
        # Read each sheet to a pandas DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        # Generate CSV file name based on sheet name
        csv_file = f'./input_data/{sheet_name}.csv'
        
        # Save the DataFrame as a CSV file
        df.to_csv(csv_file, index=False)

        print(f'Saved {sheet_name} to {csv_file}')

Saved Total Population to ./input_data/Total Population.csv
Saved Population Charts to ./input_data/Population Charts.csv
Saved Population by Age to ./input_data/Population by Age.csv
Saved Group Quarters and Households to ./input_data/Group Quarters and Households.csv
Saved Household Population by Age to ./input_data/Household Population by Age.csv
Saved Households and Housing Units to ./input_data/Households and Housing Units.csv
Saved Residential Labor Force to ./input_data/Residential Labor Force.csv
Saved Jobs to ./input_data/Jobs.csv
Saved All Data (Unformatted) to ./input_data/All Data (Unformatted).csv
Saved Variable Dictionary to ./input_data/Variable Dictionary.csv
Saved Population Chart Data to ./input_data/Population Chart Data.csv
Saved Revision History to ./input_data/Revision History.csv


In [313]:
data = pd.read_csv("./input_data/Population by Age.csv", index_col=0,low_memory=False)
# Skip the first two rows and reset the index
df_modified = data.iloc[2:].reset_index(drop=True)

# Use the first row (previously the third row of the original df) as column headers
new_header = df_modified.iloc[0] # Capture the third row's values to be the new header
df_modified = df_modified[1:] # Remove the first row from data
df_modified.columns = new_header # Set the new header as the df columns

# Reset the index again to account for the removed row
df_modified.reset_index(drop=True, inplace=True)


In [314]:
# Keep only the first 8 columns
df_filtered = df_modified.iloc[:, :8].copy()
df_filtered.to_csv(OUTPUT_TABLE1_PATH, index=False)

In [ ]:
# Take the first 16 rows of the DataFrame
df_filtered = df.iloc[:16].copy()